# 1. Preparación del Entorno
Necesitamos bitsandbytes para cargar el modelo en 4 bits, de lo contrario, los 16GB de VRAM de la T4 no serán suficientes para Llama 3.1 8B

In [ ]:
!pip install -q transformers accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.7 MB/s eta 0:00:00


# 2. Carga del Modelo (Quantized)
Utilizaremos la versión oficial de Meta o una versión "sharded" para facilitar la descarga en Colab.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import numpy as np

model_id = "unsloth/meta-llama-3.1-8b-bnb-4bit" # Versión ligera para Colab

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

# 3. El Motor de Cálculo: Perplejidad y Logprobs
Este script procesa el texto y extrae la probabilidad de cada token para calcular la métrica de "sorpresa"

In [ ]:
def analyze_text_statistics(text):
    inputs = tokenizer(text, return_tensors="pt").to("cuda")
    input_ids = inputs["input_ids"]

    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids)
        logits = outputs.logits  # [1, seq_len, vocab_size]
        loss = outputs.loss      # Negative Log Likelihood promedio

    # 1. Cálculo de Perplejidad (PPL)
    ppl = torch.exp(loss).item()

    # 2. Extracción de Logprobs para análisis estadístico
    # Shift logits y labels para alinear predicción con el siguiente token
    shift_logits = logits[..., :-1, :].contiguous()
    shift_labels = input_ids[..., 1:].contiguous()

    # Obtenemos la log-probabilidad del token real que apareció
    log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
    target_log_probs = torch.gather(log_probs, 2, shift_labels.unsqueeze(-1)).squeeze(-1)
    # Convertir a float antes de pasar a numpy
    target_log_probs = target_log_probs.detach().cpu().float().numpy()[0]

    # 3. Métrica de Sorpresa (¿Está en el Top-5?)
    top_k_indices = torch.topk(shift_logits, k=5, dim=-1).indices
    is_in_top5 = torch.any(top_k_indices == shift_labels.unsqueeze(-1), dim=-1)
    surprise_ratio = 1 - (is_in_top5.float().mean().item())

    return {
        "perplexity": ppl,
        "variance": np.var(target_log_probs),
        "surprise_ratio": surprise_ratio,
        "mean_logprob": np.mean(target_log_probs)
    }

# 4. Clasificación por Umbral (Thresholding)
Para tu TFG, deberás encontrar el "punto de corte" óptimo. Los textos de IA suelen tener una Perplejidad muy baja (predicciones muy seguras)

In [ ]:
def classify_news(stats, threshold_ppl=15.0, threshold_surprise=0.15):
    # Lógica Senior: Si la perplejidad es alta o hay mucha sorpresa, es humano
    is_human_ppl = stats["perplexity"] > threshold_ppl
    is_human_surprise = stats["surprise_ratio"] > threshold_surprise

    # Podemos usar una combinación (Votación)
    prediction = "Humano" if is_human_ppl or is_human_surprise else "IA"
    return prediction

# 5. Evaluación masiva en el Dataset
Recorre tu JSONL y guarda estas métricas. Esto será la base de tus gráficas en el TFG.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

results = []
with open("/content/drive/MyDrive/TFG/titles_data.jsonl", "r") as f:
    for line in f:
        data = json.loads(line)
        stats = analyze_text_statistics(data["title"])
        stats["label_real"] = data["is_real"]
        stats["group_id"] = data["group_id"]
        results.append(stats)

# Convertir a DataFrame para análisis de distribución
import pandas as pd
df_results = pd.DataFrame(results)
print(df_results.groupby("label_real")[["perplexity", "surprise_ratio", "variance"]].mean())

            perplexity  surprise_ratio  variance
label_real                                      
0             7.276369        0.203722  4.867878
1             9.227146        0.232583  5.959224
